In [17]:
import sys
print(sys.version)

3.10.6 (main, Oct  7 2022, 20:19:58) [GCC 11.2.0]


In [3]:
!pip3 install geopandas
!pip3 install matplotlib
!pip3 install pandas

  Obtaining dependency information for geopandas from https://files.pythonhosted.org/packages/27/27/2687abaa2ac02b5814e2929a5033da1e5d132c19a904dc56f77f63fd6eb9/geopandas-0.14.0-py3-none-any.whl.metadata
  Using cached geopandas-0.14.0-py3-none-any.whl.metadata (1.5 kB)
  Obtaining dependency information for fiona>=1.8.21 from https://files.pythonhosted.org/packages/57/b9/c3821af48b949f8db275dc5bf0d66fd4cb6a492cd96d3f427bced20945ae/Fiona-1.9.4.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached Fiona-1.9.4.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (49 kB)
  Obtaining dependency information for pyproj>=3.3.0 from https://files.pythonhosted.org/packages/f6/2b/b60cf73b0720abca313bfffef34e34f7f7dae23852b2853cf0368d49426b/pyproj-3.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached pyproj-3.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (31 kB)
  Using cached shap

In [4]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

## Get Baltimore Country Real Property Assessment data
https://opendata.maryland.gov/Business-and-Economy/Maryland-Real-Property-Assessments-Hidden-Property/ed4q-f8tm/data

Every county assesses property for tax purposes and we will be using the property's assessed value as a proxy for Sale Price which is directly correleated to loan amount on an avergage. 

In [6]:
#Baltimore County (BWC) Real Propert Assessments : apprailsals being close to real property value - assumption
#https://opendata.maryland.gov/Business-and-Economy/Maryland-Real-Property-Assessments-Hidden-Property/ed4q-f8tm/data

pd.options.display.max_columns = None
df = pd.read_csv('s3://sagemaker-us-east-1-565022919370/baltimore_county/Baltimore_County_Real_Property_Assessments__Hidden_Property_Owner_Names.csv') 
print(f"df.shape: {df.shape}") #(305196, 220)
df.head()

df.shape: (305196, 220)


,Jurisdiction Code (MDP Field: JURSCODE),County Name (MDP Field: CNTYNAME),Account ID (MDP Field: ACCTID),Real Property Search Link,FINDER Online Link,Search Google Maps for this Location,MDP Longitude (MDP Field: DIGXCORD converted to WGS84),MDP Latitude (MDP Field: DIGYCORD converted to WGS84),Mappable Latitude and Longitude,RECORD KEY: County Code (SDAT Field #1),RECORD KEY: District/Ward (SDAT Field #2),RECORD KEY: Account Number (SDAT Field #3),RECORD KEY: Subdistrict (SDAT Field #4),RECORD KEY: Geographic Code (MDP Field: GEOGCODE. SDAT Field #5),RECORD KEY: Owner Occupancy Code (MDP Field: OOI. SDAT Field #6),LEGAL DESCRIPTION: Line 1 (MDP Field: LEGAL1. SDAT Field #17),LEGAL DESCRIPTION: Line 2 (MDP Field: LEGAL2. SDAT Field #18),LEGAL DESCRIPTION: Line 3 (MDP Field: LEGAL3. SDAT Field #19),MdProperty View Edition Year (MDP Field: EXISTING),MDP Street Address (MDP Field: ADDRESS),MDP Street Address City (MDP Field: CITY),MDP Street Address Zip Code (MDP Field: ZIPCODE),MDP Street Address Units (MDP Field: STRTUNT),PREMISE ADDRESS: Number (MDP Field: PREMSNUM. SDAT Field #20),PREMISE ADDRESS: Number Suffix (SDAT Field #21),PREMISE ADDRESS: Direction (MDP Field: PREMSDIR. SDAT Field #22),PREMISE ADDRESS: Name (MDP Field: PREMSNAM. SDAT Field #23),PREMISE ADDRESS: Type (MDP Field: PREMSTYP. SDAT Field #24),PREMISE ADDRESS: City (MDP Field: PREMCITY. SDAT Field #25),PREMISE ADDRESS: Zip Code (MDP Field: PREMZIP. SDAT Field #26),PREMISE ADDRESS: Zip +4 Code (MDP Field: PREMZIP2. SDAT Field #27),PREMISE ADDRESS: Condominium Unit No. (SDAT Field #28),DEED REFERENCE 1: Clerk Initials (MDP Field: DR1CLERK. SDAT Field #29),DEED REFERENCE 1: Liber (MDP Field: DR1LIBER. SDAT Field #30),DEED REFERENCE 1: Folio (MDP Field: DR1FOLIO. SDAT Field #31),Town Code (MDP Field: TOWNCODE/DESCTOWN. SDAT Field #36),Subdivision Code (MDP Field: SUBDIVSN. SDAT Field #37),Plat Number (MDP Field: PLAT. SDAT Field #38),Section (MDP Field: SECTION. SDAT Field #39),Block (MDP Field: BLOCK. SDAT Field #40),Lot (MDP Field: LOT. SDAT Field #41),Map (MDP Field: MAP. SDAT Field #42),Grid (MDP Field: GRID. SDAT Field #43),Parcel (MDP Field: PARCEL. SDAT Field #44),Zoning Code (MDP Field: ZONING. SDAT Field #45),Exempt Class (MDP Field: EXCLASS/DESCEXCL. SDAT Field #49),MDP Street Address Type Code (MDP Field: RESITYP),Land Use Code (MDP Field: LU/DESCLU. SDAT Field #50),Multi-Parent Account Ind. (SDAT Field #55),County System Property Code (SDAT Field #56),County Service Code (SDAT Field #57),Tax Class (SDAT Field #58),Municipal Tax Class (SDAT Field #59),Recreational Influence Ind. (MDP Field: RECIND. SDAT Field #60),BPRUC (Public Use Code) (MDP Field: CIUSE/DESCCIUSE. SDAT Field #61),Ad Valorem Code (SDAT Field #62),PROPERTY FACTORS: Utilities - Water (MDP Field: PFUW. SDAT Field #63),PROPERTY FACTORS: Utilities - Sewer (MDP Field: PFUS. SDAT Field #64),PROPERTY FACTORS: Location - Waterfront (MDP Field: PFLW. SDAT Field #65),PROPERTY FACTORS: Street - Paved (MDP Field: PFSP. SDAT Field #67),PROPERTY FACTORS: Street - Unpaved (MDP Field: PFSU. SDAT Field #68),PROPERTY FACTORS: Influence - Commer/Indust. (MDP Field: PFIC. SDAT Field #69),PROPERTY FACTORS: Influence - Historical (MDP Field: PFIH. SDAT Field #70),FIELD CARD NUMBER: Field Card No. Primary (MDP Field: FCRD1ST5. SDAT Field #75),FIELD CARD NUMBER: Field Card No. Secondary (MDP Field: FCRDNXT3. SDAT Field #76),FIELD CARD NUMBER: Field Card No. Subset 3 (MDP Field: FCRDNXT2. SDAT Field #77),FIELD CARD NUMBER: Field Card No. Subset 4 (MDP Field: FCRDLST2. SDAT Field #78),SALES SEGMENT 1: Transfer Number (MDP Field: TRANSNO1. SDAT Field #79),SALES SEGMENT 1: Grantor Name (MDP Field: GRNTNAM1. SDAT Field #80),SALES SEGMENT 1: Grantor Deed Reference 1 - Clerk Initials (MDP Field: GR1CLRK1. SDAT Field #81),SALES SEGMENT 1: Grantor Deed Reference 1 - Liber (MDP Field: GR1LIBR1. SDAT Field #82),SALES SEGMENT 1: Grantor Deed Reference 1 - Folio (MDP Field: GR1FOLO1. SDAT Field #83),SALES SEGMENT 1: How C

In [7]:
#Rename BWC columns 
df.rename(columns={'CURRENT CYCLE DATA: Date Assessed (YYYY.MM) (MDP Field: LASTASSD. SDAT Field #169)': 'Current_Date_Assessed', 
                   'MDP Longitude (MDP Field: DIGXCORD converted to WGS84)': 'Longitude',
                   'MDP Latitude (MDP Field: DIGYCORD converted to WGS84)':'Latitude',
                   'CURRENT ASSESSMENT YEAR: Total Assessment (SDAT Field #172)':'Current_Total_Assessment',
                   'Land Use Code (MDP Field: LU/DESCLU. SDAT Field #50)':'Land_Use_Code',
                  }, inplace=True)

#Filter BWC dataset - remove NAas from selected columns
filtered_df = df[df['Land_Use_Code'].notna() &
                 df['Current_Total_Assessment'].notna() &
                 df['Current_Date_Assessed'].notna() &
                 df['Latitude'].notna() &
                 df['Longitude'].notna()]
filtered_df.shape #(299477, 220)

(299477, 220)

In [8]:
#Filter BWC data for current year assessments, ifnore NaNs
filtered_df = filtered_df[filtered_df['Current_Date_Assessed'].str.contains("2023.", na=False)]
filtered_df.shape

(4647, 220)

In [9]:
#Filter BWC data for current total assessments, ignore NaNs
filtered_df = filtered_df[~pd.isnull(filtered_df['Current_Total_Assessment']) &
                          ~pd.isnull(filtered_df['Longitude'])]
filtered_df.shape

(4647, 220)

In [10]:
#Filter BWC data for residential properties
filtered_df = filtered_df[filtered_df['Land_Use_Code'] == "Residential (R)"]
filtered_df.shape


(2236, 220)

In [11]:
#Are any property links duplicated
any(filtered_df['Real Property Search Link'].duplicated())

False

In [12]:
filtered_df[filtered_df['Current_Total_Assessment'].isna()]

,Jurisdiction Code (MDP Field: JURSCODE),County Name (MDP Field: CNTYNAME),Account ID (MDP Field: ACCTID),Real Property Search Link,FINDER Online Link,Search Google Maps for this Location,Longitude,Latitude,Mappable Latitude and Longitude,RECORD KEY: County Code (SDAT Field #1),RECORD KEY: District/Ward (SDAT Field #2),RECORD KEY: Account Number (SDAT Field #3),RECORD KEY: Subdistrict (SDAT Field #4),RECORD KEY: Geographic Code (MDP Field: GEOGCODE. SDAT Field #5),RECORD KEY: Owner Occupancy Code (MDP Field: OOI. SDAT Field #6),LEGAL DESCRIPTION: Line 1 (MDP Field: LEGAL1. SDAT Field #17),LEGAL DESCRIPTION: Line 2 (MDP Field: LEGAL2. SDAT Field #18),LEGAL DESCRIPTION: Line 3 (MDP Field: LEGAL3. SDAT Field #19),MdProperty View Edition Year (MDP Field: EXISTING),MDP Street Address (MDP Field: ADDRESS),MDP Street Address City (MDP Field: CITY),MDP Street Address Zip Code (MDP Field: ZIPCODE),MDP Street Address Units (MDP Field: STRTUNT),PREMISE ADDRESS: Number (MDP Field: PREMSNUM. SDAT Field #20),PREMISE ADDRESS: Number Suffix (SDAT Field #21),PREMISE ADDRESS: Direction (MDP Field: PREMSDIR. SDAT Field #22),PREMISE ADDRESS: Name (MDP Field: PREMSNAM. SDAT Field #23),PREMISE ADDRESS: Type (MDP Field: PREMSTYP. SDAT Field #24),PREMISE ADDRESS: City (MDP Field: PREMCITY. SDAT Field #25),PREMISE ADDRESS: Zip Code (MDP Field: PREMZIP. SDAT Field #26),PREMISE ADDRESS: Zip +4 Code (MDP Field: PREMZIP2. SDAT Field #27),PREMISE ADDRESS: Condominium Unit No. (SDAT Field #28),DEED REFERENCE 1: Clerk Initials (MDP Field: DR1CLERK. SDAT Field #29),DEED REFERENCE 1: Liber (MDP Field: DR1LIBER. SDAT Field #30),DEED REFERENCE 1: Folio (MDP Field: DR1FOLIO. SDAT Field #31),Town Code (MDP Field: TOWNCODE/DESCTOWN. SDAT Field #36),Subdivision Code (MDP Field: SUBDIVSN. SDAT Field #37),Plat Number (MDP Field: PLAT. SDAT Field #38),Section (MDP Field: SECTION. SDAT Field #39),Block (MDP Field: BLOCK. SDAT Field #40),Lot (MDP Field: LOT. SDAT Field #41),Map (MDP Field: MAP. SDAT Field #42),Grid (MDP Field: GRID. SDAT Field #43),Parcel (MDP Field: PARCEL. SDAT Field #44),Zoning Code (MDP Field: ZONING. SDAT Field #45),Exempt Class (MDP Field: EXCLASS/DESCEXCL. SDAT Field #49),MDP Street Address Type Code (MDP Field: RESITYP),Land_Use_Code,Multi-Parent Account Ind. (SDAT Field #55),County System Property Code (SDAT Field #56),County Service Code (SDAT Field #57),Tax Class (SDAT Field #58),Municipal Tax Class (SDAT Field #59),Recreational Influence Ind. (MDP Field: RECIND. SDAT Field #60),BPRUC (Public Use Code) (MDP Field: CIUSE/DESCCIUSE. SDAT Field #61),Ad Valorem Code (SDAT Field #62),PROPERTY FACTORS: Utilities - Water (MDP Field: PFUW. SDAT Field #63),PROPERTY FACTORS: Utilities - Sewer (MDP Field: PFUS. SDAT Field #64),PROPERTY FACTORS: Location - Waterfront (MDP Field: PFLW. SDAT Field #65),PROPERTY FACTORS: Street - Paved (MDP Field: PFSP. SDAT Field #67),PROPERTY FACTORS: Street - Unpaved (MDP Field: PFSU. SDAT Field #68),PROPERTY FACTORS: Influence - Commer/Indust. (MDP Field: PFIC. SDAT Field #69),PROPERTY FACTORS: Influence - Historical (MDP Field: PFIH. SDAT Field #70),FIELD CARD NUMBER: Field Card No. Primary (MDP Field: FCRD1ST5. SDAT Field #75),FIELD CARD NUMBER: Field Card No. Secondary (MDP Field: FCRDNXT3. SDAT Field #76),FIELD CARD NUMBER: Field Card No. Subset 3 (MDP Field: FCRDNXT2. SDAT Field #77),FIELD CARD NUMBER: Field Card No. Subset 4 (MDP Field: FCRDLST2. SDAT Field #78),SALES SEGMENT 1: Transfer Number (MDP Field: TRANSNO1. SDAT Field #79),SALES SEGMENT 1: Grantor Name (MDP Field: GRNTNAM1. SDAT Field #80),SALES SEGMENT 1: Grantor Deed Reference 1 - Clerk Initials (MDP Field: GR1CLRK1. SDAT Field #81),SALES SEGMENT 1: Grantor Deed Reference 1 - Liber (MDP Field: GR1LIBR1. SDAT Field #82),SALES SEGMENT 1: Grantor Deed Reference 1 - Folio (MDP Field: GR1FOLO1. SDAT Field #83),SALES SEGMENT 1: How Conveyed Ind. (MDP Field: CONVEY1. SDAT Field #87),SALES SEGMENT 1: Transfer Date (YYYY.MM.DD) (MDP Field: TRADATE. SDAT Field #89

In [13]:
filtered_df.shape

(2236, 220)

In [16]:
filtered_df.to_csv("s3://sagemaker-us-east-1-565022919370/baltimore_county/BMC_filtered_df.csv");